# Notebook version of Main python file that will be used to create visualization tool

The final tool will likely be a more concise .py file,  but this notebook should be helpful for creating the tool and debugging

## Initial Script

In [1]:
import pandas as pd
import numpy as np
from datetime import date
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models.tickers import FixedTicker
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs, TableColumn, DataTable, RadioGroup,RadioButtonGroup, Dropdown,DateRangeSlider
from bokeh.models import ColumnDataSource, DataTable, TableColumn
from bokeh.layouts import WidgetBox


# from bokeh.io import output_file
from bokeh.io import output_notebook, curdoc
from bokeh.layouts import column, row
from bokeh.models import Button, ColumnDataSource, Slider, TextInput
from bokeh.plotting import figure
from bokeh.models.widgets import DateRangeSlider


from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs, TableColumn, DataTable, RadioGroup,RadioButtonGroup, Dropdown

from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()


from bokeh.io import show
from bokeh.plotting import figure









Loading BokehJS ...

In [2]:
#load data

allData = pd.read_csv("C:/Users/Caleb/Desktop/Poolla Lab/Data/15min_EV_PV_homes_only.csv") # Load all the data

                          

In [3]:
filterData = allData[["car1","grid","solar","local_15min","dataid","state"]] # cutting down nonessential columns for the sake of runtime
filterData = filterData.rename(columns={"local_15min":"time"}) 
filterData["time"] = pd.to_datetime(filterData["time"]) # change to appropriate data type



# Main


In [4]:

# Grid:
# eGauge data present measuring power drawn from the electrical grid

def modify_doc(doc):

    #all_min_date = filterData.groupby('dataid').agg(min)["time"]
    #all_max_date = filterData.groupby('dataid').agg(max)["time"]

    dummy_daterange = ['2019-05-01', '2019-08-20']
    dummy_granularity = '15 Minutes'
    dummy_house = 5679

    def plot3_data(daterange = dummy_daterange, xaxis = dummy_granularity):

        houseData = filterData[filterData['state'] == 'NY'][['time','grid']]
        # Resample doesn't like multiple time zones, note to find a way around this
        houseData = houseData.sort_values('time', ascending = True)
        houseData.index = houseData['time']
        houseData = houseData.loc[daterange[0]:daterange[1],:] # cut to the days requested
       
        if xaxis == '15 Minutes':
            houseData = houseData.drop(columns="time")
            houseData['grid'] = houseData['grid'] * 60 * 15 / 3600 # kWh

        if xaxis == 'Hour':
            houseData['grid'] = houseData['grid'] * 60 * 15 / 3600 # kWh
            houseData = houseData.resample('1h').sum()

        if xaxis == 'Day':
            houseData['grid'] = houseData['grid'] * 60 * 15 / 3600 # kWh
            houseData = houseData.resample('1d').sum()

        if xaxis == 'Week':
            houseData['grid'] = houseData['grid'] * 60 * 15 / 3600 # kWh
            houseData = houseData.resample('1w').sum()

        if xaxis == 'Month':
            houseData['grid'] = houseData['grid']* 60 * 15 / 3600 # kWh
            houseData = houseData.resample('1m').sum()

        houseData['time'] = houseData.index
        netLoad = houseData.groupby(houseData['time'])['grid'].sum() # L - G
        # note that all of the houses do not have data taken for the same range
        # This affects the summation excluding some houses at different points in time
        price = netLoad > 0
        price = pd.DataFrame(data = price)
        
        return ColumnDataSource(price)

    def plot4_data(daterange = dummy_daterange, house = dummy_house):
        sortedData = filterData[filterData['state'] == 'NY'][['time','grid','solar','dataid']].sort_values('time', ascending = True)
        sortedData.index = sortedData['time']
        sortedData = sortedData.loc[daterange[0]:daterange[1],:]
        sortedData['grid'] = sortedData['grid'] * 60 * 15 / 3600 # kWh
        sortedData['solar'] = sortedData['solar'] * 60 * 15 / 3600 # kWh

        houseData = sortedData[sortedData['dataid'] == house]

        L = houseData['grid'] + houseData['solar'] # (L-S) + S = L
        S =  houseData['solar'] # S
        Sbar = ( S < L ) * S + ( S > L ) * L # Sbar
        solarAtDiscount = S - Sbar

        Load = L.sum() # blue plot no share
        SelfSolarSum = Sbar.sum() # green plot no share
        DiscountSum = solarAtDiscount.sum() # red plot no share

        sumL = L.cumsum()
        sumS = S.cumsum()
        Shat = ( sumS < sumL ) * S + ( sumS > sumL ) * (S * sumL/sumS)

        Load = L.sum() # blue plot share
        CommunitySolarSum = Shat.sum() # green plot share
        DiscountShareSum = (S*Shat).sum() # red plot share

        d = {'blue no share': Load, 'green no share': SelfSolarSum, 'red no share': DiscountSum,
                'blue share': Load, 'green share': CommunitySolarSum, 'red share': DiscountShareSum}

        #d = {'Sharing $': [eqCost], 'Normal $': [cost],'Saved':[cost-eqCost]}
        df = pd.DataFrame([[Load,SelfSolarSum,DiscountSum,Load,CommunitySolarSum,DiscountShareSum]],
                columns = ['blue no share','green no share','red no share',
                     'blue share', 'green share','red share'])

        print(df)
        print(ColumnDataSource(df))
        return ColumnDataSource(df)

    def plot4_plot(src):  #plot the new plot 4
        plot4 = figure(title='Plot 1: To share or not to share',
                       x_axis_label='Without Sharing',
                       y_axis_label='Energy Consumed [kWh]')

        plot4.plot_width = 600
        plot4.plot_height = 400
        plot4.vbar(x='columns', top='data', width=1, source=src)

        return plot4
        

    def plot3_plot(src):
        plot3 = figure(title = 'Equilibrium Price',x_axis_type="datetime", x_axis_label="Time",
            y_axis_label="Price")
        plot3.line('time','grid',source = src)
        plot3.yaxis.ticker = [0,1]
        plot3.yaxis.major_label_overrides = {0: 'Pi_nm', 1: 'Pi_g'}

        return plot3


    def update(attr, old, new):
        
        daterange_to_plot = ['2019-05-01', '2019-08-20']

        daterange_raw = list(date_range_slider.value_as_datetime)
        daterange_to_plot = [daterange_raw[0].strftime("%Y-%m-%d"), daterange_raw[1].strftime("%Y-%m-%d")]
        
        home_id_to_plot = int(home_id_selector.value)

        granularity_to_plot = granularity_1.labels[granularity_1.active]
        
        new_src3 = plot3_data(daterange = daterange_to_plot, xaxis = granularity_to_plot)
        new_src4 = plot4_data(daterange = daterange_to_plot, house = home_id_to_plot)

        src3.data.update(new_src3.data)
        src4.data.update(new_src4.data)
    
    ## Granularity Button
    granularity_1 = RadioGroup(
        labels=["15 Minutes", "Hour", "Day", "Week", "Month"], active=0,
            background ='paleturquoise',
            max_width = 100)
    granularity_1.on_change('active',
                            update)
    
    ## Daterange Slider Button
    date_range_slider = DateRangeSlider(title="Date Range: ", 
            start=date(2019, 5, 1), end=date(2019, 8, 20),value=(date(2019, 5, 1),
                date(2019, 8, 20)), step=1, callback_policy = 'mouseup',max_width = 250)
    date_range_slider.on_change("value_throttled", update)

    ## Home Selector
    home_ids_available = np.unique(filterData[filterData['state'] == 'NY']['dataid'])

    home_ids_available = list(map(str, home_ids_available))
    home_id_selector = Dropdown(label="Home ID", button_type="warning", menu=home_ids_available, value="27", max_width = 350)
    home_id_selector.on_change('value',update)


    ## Initialize src and plot
    src3 = plot3_data(['2019-05-01', '2019-08-20'],'15 Minutes')
    src4 = plot4_data(['2019-05-01', '2019-08-20'],5679)

    plot3 = plot3_plot(src3)
    plot4 = plot4_plot(src4)

    ## Table
   # columns = [
   #         TableColumn(field='Sharing $', title='Sharing Cost'),
   #         TableColumn(field='Normal $', title='Regular Cost'),
   #         TableColumn(field='Saved',title='Amount Saved')
   #         ]
   # data_table = DataTable(source=src4,columns = columns,width=350, height=280)


    # Create a layout
    controls = WidgetBox(column(row(granularity_1,date_range_slider),
        home_id_selector), sizing_mode = 'scale_both')

    layout = row(controls,plot4)

    # Make a tab with the layout
    tab = Panel(child=layout, title='Market Analysis')
    tabs = Tabs(tabs=[tab])
    doc.add_root(tabs)




    controls = WidgetBox(granularity_1,home_id_selector, date_range_slider)  #data_type_selector)


    

    
    
    
#     plot2=plot2_plot(src)    
#     show(plot2)


#stuff to make the app run in this notebook (not part of final .py)
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [9]:
def modify_doc(doc): #this whole modify_doc container is used to make app run in this notebook(not part of final .py)

    # dummy data that will be replaced by button values once we get those implemented (right now only granulaity button is implemented)

    all_min_date = filterData.groupby('dataid').agg(min)["time"]
    all_max_date = filterData.groupby('dataid').agg(max)["time"]

    dummy_daterange = ['2019-05-01', '2019-08-20']
    dummy_home_id = 27
    dummy_data_type = 'car1'
    dummy_granularity = '15 Minutes'

    def plot1_data(house, daterange=dummy_daterange, data=dummy_data_type, xaxis=dummy_granularity):

        # house is an integer number ex. 27
        # daterange is an array with 2 strings, start date and end date. ex. ['2019-05-01','2019-08-09']
        # weekdays is an ordered list 0-6 of integers ex. [1,4,6] (these are the days we want to exclude)
        # data is a string ex. 'car1'
        # xaxis is also a string ex. 'hour'

        houseData = filterData[filterData['dataid'] == house].sort_values('time', ascending=True)[[data, 'time']]
        # that cuts the house, sorts by ascending time, and pulls out only the type of data that was requested
        houseData.index = houseData['time']  # reindex by the datetime
        houseData = houseData.loc[daterange[0]:daterange[1], :]  # cut to the days requested

        # Now we get into the xaxis user options #
        if xaxis == '15 Minutes':
            houseData = houseData.drop(columns="time")

        if xaxis == 'Hour':
            houseData = houseData.resample('1h').mean()

        if xaxis == 'Day':
            houseData = houseData.resample('1d').mean()

        if xaxis == 'Week':
            houseData = houseData.resample('1w').mean()

        if xaxis == 'Month':
            houseData = houseData.resample('1m').mean()
        # if none of these, 15 Minutes is implied and passed through
        return ColumnDataSource(houseData)

    ####for now we will ignore the same method for plot #2

    #     def plot2_data(house,daterange,weekdays,data,xaxis):
    #         houseData = filterData[filterData['dataid'] == house].sort_values('time', ascending = True)[[data,'time']]
    #         # that cuts the house, sorts by ascending time, and pulls out only the type of data that was requested
    #         houseData.index = houseData['time'] # reindex by the datetime
    #         houseData = houseData.loc[daterange[0]:daterange[1],:] # cut to the days requested

    #         for i in weekdays:
    #             houseData = houseData[houseData['time'].dt.dayofweek != i] # cut out days we dont want

    #         if xaxis == 'avgday':
    #             houseData = houseData.resample('1d').sum() ####chjange to mean
    #             houseData['time'] = houseData.index
    #             houseData = houseData.groupby(houseData['time'].dt.dayofweek)[data].mean()
    #             houseData.index = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

    #         if xaxis == 'avghour':
    #             houseData = houseData.resample('1h').mean()
    #             houseData['time'] = houseData.index
    #             houseData = houseData.groupby(houseData['time'].dt.hour)[data].mean() # Does not account for UTC change!

    #         return houseData #ColumnDataSource(houseData)

    # now we need to set up our plot axis

    ##########method2: create plots

    def plot1_plot(src):
        # create the plot every time  a change occurs
        plot1 = figure(title="PLOT 1", x_axis_type="datetime", x_axis_label="Local Time",
                       y_axis_label="Consumption [kW]")
        plot1.line('time', "grid", source=src, )  # simple line plot   #data_type_available[data_type_selector.active]

        return plot1  # plot object type

    #     def plot2_plot(src):
    #         plot2 = figure(title="PLOT 2", x_axis_type="datetime",x_axis_label="Local Time", y_axis_label="Consumption [kW]")
    #         plot2.line('time','grid',source=src,)
    #         return plot2

    #########Method3: Update App

    def update(attr, old, new):  # still a little unsure how the update function gets values passed in implicitly
        # these values to be replaced with button/user inputs
        
        home_id_to_plot = int(home_id_selector.value)
        print(attr,old,new)
        
#         if home_id_selector.value == new:
            
#             daterange_to_plot = [all_min_date[home_id_to_plot].strftime("%Y-%m-%d"), all_max_date[home_id_to_plot].strftime("%Y-%m-%d")]
#             date_range_slider.start=all_min_date[home_id_to_plot]
#             date_range_slider.end=all_max_date[home_id_to_plot]
#             date_range_slider.value=(all_min_date[home_id_to_plot],all_max_date[home_id_to_plot])

#             print("if 1 truth")
#         else:
        daterange_raw = list(date_range_slider.value_as_datetime)
        daterange_to_plot = [daterange_raw[0].strftime("%Y-%m-%d"), daterange_raw[1].strftime("%Y-%m-%d")]
#         print("falsyy")

        date_range_slider.update(start=all_min_date[home_id_to_plot], end=all_max_date[home_id_to_plot],value=(all_min_date[home_id_to_plot],all_max_date[home_id_to_plot]))

        
#         print("after:",new)
        
        
#         print(attr,old,new)

#         daterange_to_plot=list(string_date1,string_date2)

#         daterange_to_plot=dummy_daterange

        data_type_to_plot = "grid"                     #data_type_available[data_type_selector.active]


        # only working button call so far
        granularity_to_plot = granularity_1.labels[granularity_1.active]

        #         granularity_to_plot= [granularity_1.labels[i] for i in granularity_1.active]
        print("before data create")
        # create a new data source
        new_src1 = plot1_data(home_id_to_plot, daterange=daterange_to_plot, data=data_type_to_plot,
                              xaxis=granularity_to_plot)
        print("before data update")
        # push new data  to the source data the rest of the app is usig for plot1
        src1.data.update(new_src1.data)
        print("updated data")
        
#     def new_home(attr,old,new):
#         return
    
    
    ############# Add widgets

    # only the granularity implemented so far
    granularity_1 = RadioGroup(
        labels=["15 Minutes", "Hour", "Day", "Week", "Month"], active=0)

    granularity_1.on_change('active',
                            update)  # not sure exactly how this works but runs update on the change of the button and passes through the value of the button

    home_ids_available = np.unique(filterData['dataid'])
    home_ids_available= list(map(str, home_ids_available))
    home_id_selector = Dropdown(label="Home ID to Plot", button_type="warning", menu=home_ids_available,value="27")

    home_id_selector.on_change('value',update)



    date_range_slider = DateRangeSlider(title="Date Range: ", start=date(2019,5,1), end=date(2019, 8, 20), value=(date(2019,5,1), date(2019, 8, 20)), step=1)
    date_range_slider.on_change("value", update)

    # data_type_selector = RadioButtonGroup(labels=["Net Home Consumption","Solar Generation","EV Consumption"],active=0)

    # data_type_selector.on_change('active',update)
    # data_type_available=["grid","solar","car1"]



    ############ Initialize opening plot and data
    src1 = plot1_data(int(home_ids_available[0]), ['2019-05-01', '2019-08-20'], 'grid', '15 Minutes')  # start with a data range we know is correct
    plot1 = plot1_plot(src1)

    ##### Formatting of the app screen

    # Put controls in a single element (add more later to format)
    controls = WidgetBox(granularity_1,home_id_selector, date_range_slider)  #data_type_selector)

    # Create a row layout
    layout = row(controls, plot1)
    
    tab = Panel(child=layout, title='First Tab')
    tabs = Tabs(tabs=[tab])
    doc.add_root(tabs)
    
    
    
#     plot2=plot2_plot(src)    
#     show(plot2)


#stuff to make the app run in this notebook (not part of final .py)
handler = FunctionHandler(modify_doc)
app = Application(handler)



In [5]:
show(app, 'localhost:8889')



ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "top" value "data", key "x" value "columns" [renderer: GlyphRenderer(id='1087', ...)]


   blue no share  green no share  red no share  blue share  green share  \
0      2712.4835       934.14075      1226.138   2712.4835  2160.185003   

     red share  
0  1412.021076  
ColumnDataSource(id='1007', ...)


In [7]:


########method1: create data source for plots

# dummy data that will be replaced by button values once we get those implemented (right now only granulaity button is implemented)
dummy_daterange = ['2019-05-01', '2019-08-20']
dummy_home_id = 27
dummy_data_type = 'grid'
dummy_granularity = '15 Minutes'


def plot1_data(house,daterange = dummy_daterange,data= dummy_data_type,xaxis = dummy_granularity):

    # house is an integer number ex. 27
    # daterange is an array with 2 strings, start date and end date. ex. ['2019-05-01','2019-08-09']
    # weekdays is an ordered list 0-6 of integers ex. [1,4,6] (these are the days we want to exclude)
    # data is a string ex. 'car1'
    # xaxis is also a string ex. 'hour'

    houseData = filterData[filterData['dataid'] == house].sort_values('time', ascending=True)[[data, 'time']]
    # that cuts the house, sorts by ascending time, and pulls out only the type of data that was requested
    houseData.index = houseData['time']  # reindex by the datetime
    houseData = houseData.loc[daterange[0]:daterange[1], :]  # cut to the days requested

    # Now we get into the xaxis user options #
    if xaxis == '15 Minutes':
        houseData = houseData.drop(columns="time")

    if xaxis == 'Hour':
        houseData = houseData.resample('1h').mean()
       

    if xaxis == 'Day':
        houseData = houseData.resample('1d').mean()
       

    if xaxis == 'Week':
        houseData = houseData.resample('1w').mean()
        

    if xaxis == 'Month':
        houseData = houseData.resample('1m').mean()
        
    # if none of these, 5 Minutes is implied and passed through
    return houseData


In [8]:
plot1_data(6139,dummy_daterange,"grid","Month")

TypeError: '<' not supported between instances of 'datetime.datetime' and 'str'

In [9]:
# x=type(zz2["car1"][269474])
id=661
zz=filterData[filterData["dataid"]==id]
zz2=zz[zz["solar"]==0]

# min_t=zz["time"].sort(ascending=True)[0]


zzz1=filterData[filterData['dataid'] == id].sort_values('time', ascending=False)

max1=zzz1['time'].values[0]

zzz2=filterData[filterData['dataid'] == id].sort_values('time', ascending=True)
min2=zzz2['time'].values[0]

print([min2,max1])

# print(zzz1,zzz2)

[datetime.datetime(2018, 1, 1, 0, 0, tzinfo=tzoffset(None, -21600)), datetime.datetime(2018, 12, 31, 23, 45, tzinfo=tzoffset(None, -21600))]


In [10]:
filterData["time"].apply(type).unique

<bound method Series.unique of 0         <class 'datetime.datetime'>
1         <class 'datetime.datetime'>
2         <class 'datetime.datetime'>
3         <class 'datetime.datetime'>
4         <class 'datetime.datetime'>
                     ...             
384224    <class 'datetime.datetime'>
384225    <class 'datetime.datetime'>
384226    <class 'datetime.datetime'>
384227    <class 'datetime.datetime'>
384228    <class 'datetime.datetime'>
Name: time, Length: 384229, dtype: object>

In [91]:
all_min_date=filterData.groupby('dataid').agg(min)["time"]


all_max_date=filterData.groupby('dataid').agg(max)["time"]

In [92]:
allmin[3000]


datetime.datetime(2019, 5, 1, 0, 0, tzinfo=tzoffset(None, -18000))

In [93]:
allmax

dataid
27      2019-10-31 23:45:00-05:00
661     2018-12-31 23:45:00-06:00
1222    2019-10-31 23:45:00-05:00
1642    2018-12-31 23:45:00-06:00
2335    2018-12-31 23:45:00-06:00
3000    2019-10-31 23:45:00-05:00
4373    2018-12-31 23:45:00-06:00
4767    2018-12-31 04:45:00-06:00
5679    2019-10-31 23:45:00-05:00
6139    2018-12-31 23:45:00-06:00
7719    2018-12-31 23:45:00-06:00
8156    2018-12-31 23:45:00-06:00
9278    2018-12-31 23:45:00-06:00
Name: time, dtype: object

In [94]:
testData.resample('1w').sum()

NameError: name 'testData' is not defined

In [95]:
list(np.unique(filterData['dataid'].values))


[27, 661, 1222, 1642, 2335, 3000, 4373, 4767, 5679, 6139, 7719, 8156, 9278]

In [96]:
xx=np.unique(filterData['dataid'])
xxx=list(map(str, xx))
print(xx)
print(xxx)

[  27  661 1222 1642 2335 3000 4373 4767 5679 6139 7719 8156 9278]
['27', '661', '1222', '1642', '2335', '3000', '4373', '4767', '5679', '6139', '7719', '8156', '9278']


In [97]:
    
    
    home_ids_available = np.unique(filterData['dataid'])
    home_ids_available= list(map(str, home_ids_available))
    home_id_selector = Dropdown(label="Home ID to Plot", button_type="warning", menu=home_ids_available, value="25")

    show(home_id_selector)
    print(int(home_id_selector.value))

25


In [98]:
from bokeh.models.widgets import RangeSlider,DateSlider
from datetime import date

In [99]:
slides=DateSlider(title="Date Range:",start=date(2017, 1, 1), end=date.today(), value=date(2017, 9, 7), step=1)
show(slides)

In [100]:
from datetime import date

from bokeh.models.widgets import DateRangeSlider
from bokeh.layouts import layout

from bokeh.io import curdoc

date_range_slider = DateRangeSlider(title="Date Range: ", start=date(2019,5,1), end=date(2019, 8, 20), value=(date(2019,5,1), date(2019, 8, 20)), step=1)

l = layout(children=[[date_range_slider]])
curdoc().add_root(l)
curdoc().title = "DateRangeSlider Example"

cc=list(date_range_slider.value)
cc2=[cc[0].strftime("%Y-%m-%d"), cc[1].strftime("%Y-%m-%d")]
show(date_range_slider)

In [154]:
home_id_to_plot=661

date_range_slider.update(start=all_min_date[home_id_to_plot], end=all_max_date[home_id_to_plot],value=(all_min_date[home_id_to_plot],all_max_date[home_id_to_plot]))

show(date_range_slider)


In [107]:
all_min_date

dataid
27      2019-05-01 00:00:00-05:00
661     2018-01-01 00:00:00-06:00
1222    2019-05-01 00:00:00-05:00
1642    2018-01-01 00:00:00-06:00
2335    2018-01-01 00:00:00-06:00
3000    2019-05-01 00:00:00-05:00
4373    2018-01-01 00:00:00-06:00
4767    2018-01-01 00:00:00-06:00
5679    2019-05-01 00:00:00-05:00
6139    2018-01-01 00:00:00-06:00
7719    2018-01-01 00:00:00-06:00
8156    2018-01-01 00:00:00-06:00
9278    2018-01-01 00:00:00-06:00
Name: time, dtype: object

In [22]:
daterange_to_plot = list(date_range_slider.value)
daterange_to_plot = [daterange_to_plot[0].strftime("%Y-%m-%d"), daterange_to_plot[1].strftime("%Y-%m-%d")]


In [23]:
daterange_to_plot

['2019-05-01', '2019-08-20']

In [24]:
# daterange=['2019-05-01','2019-08-20']
daterange=list(date_range_slider.value)

house=27
data="grid"

houseData = filterData[filterData['dataid'] == house].sort_values('time', ascending=True)[[data, 'time']]
# that cuts the house, sorts by ascending time, and pulls out only the type of data that was requested
houseData.index = houseData['time']  # reindex by the datetime
houseData = houseData.loc[daterange[0]:daterange[1], :]  # cut to the days requested


In [25]:
houseData

,grid,time
time,,
2019-05-01 00:00:00-05:00,0.997,2019-05-01 00:00:00-05:00
2019-05-01 00:15:00-05:00,0.750,2019-05-01 00:15:00-05:00
2019-05-01 00:30:00-05:00,0.608,2019-05-01 00:30:00-05:00
2019-05-01 00:45:00-05:00,1.057,2019-05-01 00:45:00-05:00
2019-05-01 01:00:00-05:00,0.518,2019-05-01 01:00:00-05:00
...,...,...
2019-08-19 23:00:00-05:00,0.747,2019-08-19 23:00:00-05:00
2019-08-19 23:15:00-05:00,0.443,2019-08-19 23:15:00-05:00
2019-08-19 23:30:00-05:00,0.185,2019-08-19 23:30:00-05:00


In [26]:
cc

[datetime.date(2019, 5, 1), datetime.date(2019, 8, 20)]

In [27]:
string_date=str(cc[0].year)+"-"+str(cc[0].month)+"-"+str(cc[0].day)
string_date

'2019-5-1'

In [28]:
cc[0].strftime("%Y-%m-%d")

'2019-05-01'

In [49]:
        daterange_raw = list(date_range_slider.value)
        # daterange_to_plot = [daterange_raw[0].strftime("%Y-%m-%d"), daterange_raw[1].strftime("%Y-%m-%d")]

        string_date1 = str(daterange_raw[0].year) + "-" + str(daterange_raw[0].month) + "-" + str(daterange_raw[0].day)
        string_date2 = str(daterange_raw[1].year) + "-" + str(daterange_raw[1].month) + "-" + str(daterange_raw[1].day)
        
        daterange_to_plot=list([string_date1,string_date2])

In [52]:
type(daterange_to_plot)

list

In [51]:
type(['2019-05-01', '2019-08-20'])

list

In [53]:
daterange_to_plot== ['2019-05-01', '2019-08-20']

False

In [59]:
"{:%Y-%m-%d}".format(cc[0])

'2019-05-01'

In [64]:
pd.to_datetime(1566227060589)

Timestamp('1970-01-01 00:26:06.227060589')

NameError: name 'datetime' is not defined

In [75]:
pd.to_datetime(1558115073488)

Timestamp('1970-01-01 00:25:58.115073488')

In [85]:
allmin[27]

Timestamp('2019-05-01 00:00:00-0500', tz='tzoffset(None, -18000)')

In [145]:
date_range_slider.update(start=all_min_date[home_id_to_plot], end=all_max_date[home_id_to_plot],value=(all_min_date[home_id_to_plot],all_max_date[home_id_to_plot]))
date_range_slider.value=("2017-01-01","2019-01-01")


In [146]:
show(date_range_slider)

In [150]:
date_range_slider.properties_containers

<bound method HasProps.properties_containers of <class 'bokeh.models.widgets.sliders.DateRangeSlider'>>